# Lab 1: Classification, some n-gram math and tf-idf

Due date: Jan 29th, 2024, 11:59 PM

Agenda
------
+ Detecting the end of a sentence
    - Rule-based classifier
+ Detecting the sentiment of a sentence
    - Rule-based classifier (counting words)
    - Measuring Accuracy, Precision, Recall (evaluating a classifier)
+ N-gram Math
+ tf-idf


Classification can take several forms. Here are some vocabulary terms to get you started:

- __classifier__: a model that takes data (text, in NLP) as input and outputs a category
- __binary classification__: a model that takes input and outputs *one of two* categories (e.g. "positive" or "negative")
- __multinomial classification__: a model that takes input and outputs *one of many* categories (e.g. "positive", "neutral" or "negative" or a language model that chooses one token from the entire vocabulary)


- __rule-based classifier__: a classifier that functions based on rules that humans come up with (e.g. "the end of a sentence is when there is a "." ")
- __statistical classifier__: a classifier that functions based on counts (statistics) that it has gathered or based on running an algorithm to automatically train parameters on a given data set.
    
In this lab, you'll be building rule-based classifiers and evaluating them. We'll learn about our first statistical classifier next lecture.


# Task 0: Who is in your group?

__Nikita Mandal__

__Sanjana JD__

__Ashutosh Rane__

Task 1: Tokenizer
-------


In [98]:

def tokenize(s: str) -> list:
    """
    Tokenize a string based on whitespace
    Parameters:
        s - string piece of text
    returns a list of strings from the text.
    Each item is an individual linguistic unit.
    """
    # TODO: fill in this function
    # you may find using the str.split() functions
    # helpful
    # you may make whatever decisions make sense to you/your
    # group about how to tokenize.
    # don't spend longer than ~5 minutes implementing this function.
    return s.split()

test_string = "Hello! How are you doing ?"
tokenized = tokenize(test_string)
print(tokenized)

['Hello!', 'How', 'are', 'you', 'doing', '?']


1. What decisions does your tokenizer make about what should/should not be a token? __It seperates the tokens based on white space. If there is a white space in between two words/punctuation, it is considered as an independent token.__

In [99]:
# read in the text of moby dick (ensure the txt file is in the same directory as this notebook)
# if you do not already have - link to download text http://www.gutenberg.org/files/2701/2701-0.txt
# right click and 'save as' into the directory this notebook is located as 'moby_dick.txt'
moby = open('mobydick.txt', "r", encoding='utf-8')

print(moby.readline()) # first line is blank
print(moby.readline()) # second line just to see if its correct
moby.close()

# now read in the full contents
moby = open('mobydick.txt', "r", encoding='utf-8')
contents = moby.read()
moby.close()

print("The length is: ",len(contents)) # how long is this string?

The Project Gutenberg eBook of Moby-Dick; or The Whale, by Herman Melville



The length is:  1238355


In [100]:
# call your tokenize function on the contents of moby dick
tokens = tokenize(contents)
print(tokens)
print("Token count:", len(tokens))
print("Vocabulary count: ", len(set(tokens)))

['\ufeffThe', 'Project', 'Gutenberg', 'eBook', 'of', 'Moby-Dick;', 'or', 'The', 'Whale,', 'by', 'Herman', 'Melville', 'This', 'eBook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'United', 'States', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever.', 'You', 'may', 'copy', 'it,', 'give', 'it', 'away', 'or', 're-use', 'it', 'under', 'the', 'terms', 'of', 'the', 'Project', 'Gutenberg', 'License', 'included', 'with', 'this', 'eBook', 'or', 'online', 'at', 'www.gutenberg.org.', 'If', 'you', 'are', 'not', 'located', 'in', 'the', 'United', 'States,', 'you', 'will', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using', 'this', 'eBook.', 'Title:', 'Moby-Dick;', 'or', 'The', 'Whale', 'Author:', 'Herman', 'Melville', 'Release', 'Date:', 'June,', '2001', '[eBook', '#2701]', '[Most', 'recently', 'updated:', 'August', '18,', '2021]'

2. How many tokens are in *Moby Dick* when you use your `tokenize` function on its contents? __Token count: 215864__
3. How big is the __vocabulary__ of *Moby Dick* when you use your `tokenize` function on its contents? __Vocabulary count:  33569__

# Task 2: Detecting the end of a sentence


A classifier is, in essence, a function that takes some data $x$ and assigns some label $y$ to it. For a binary classifier, we can model this a function that takes a data point $x$ and returns either `True` or `False`.

Later in this class we'll learn about how to build classifiers that automatically learn how to do this, but we'll start where NLP started—writing some rule-based classifiers.

In [101]:
def classify_sentence_end(text: str, target_index: int) -> bool:
    """
    Classify whether or not a *location* is the end of a sentence within
    a given text
    Parameters:
        text - string piece of text
        target_index - int candidate location
    returns true if the target index is the end of a sentence.
    False otherwise.
    """
    # TODO: write a simple, rule-based classifier that
    # decides whether or not a specific location is the
    # end of a sentence
    sentence_endings = {'.', '!', '?'}
    return text[target_index] in sentence_endings
    

# look at the code in the cell below to see example usage

In [102]:
# example text
# feel free to go through different examples

# This is the given example text
"""Stocks were up as advancing issues outpaced declining issues
          on the NYSE by 1.5 to 1. Large- and small-cap stocks were both strong,
          while the S.&P. 500 index gained 0.46% to finish at 2,457.59. Among
          individual stocks, the two top percentage gainers in the S.&P. 500
          were Incyte Corporation and Gilead Sciences Inc."""

example = "Stocks were up as advancing issues outpaced declining issues on the NYSE by 1.5 to 1. Large- and small-cap stocks were both strong, while the S.&P. 500 index gained 0.46% to finish at 2,457.59. Among individual stocks, the two top percentage gainers in the S.&P. 500 were Incyte Corporation and Gilead Sciences Inc."

# this code will go through and
# build up a string based on the sentence
# decisions that your classifier comes up with
# it will put "****" between the sentences
# you do not need to modify any code here
so_far = ""
sentence_count = 0

for index in range(len(example)):
    result = classify_sentence_end(example, index)
    so_far += example[index]

    if result:
        sentence_count += 1
        print(so_far)
        print("****")
        so_far = ""

print(f"Total number of sentences: {sentence_count}")

Stocks were up as advancing issues outpaced declining issues on the NYSE by 1.
****
5 to 1.
****
 Large- and small-cap stocks were both strong, while the S.
****
&P.
****
 500 index gained 0.
****
46% to finish at 2,457.
****
59.
****
 Among individual stocks, the two top percentage gainers in the S.
****
&P.
****
 500 were Incyte Corporation and Gilead Sciences Inc.
****
Total number of sentences: 10


1. How many sentences are detected using your end of sentence classifier? __10__
2. Where did your end of sentence classifier make a mistake? __It counted the decimal points or a period used after an abbreviation also as a sentence ending.__

Task 3: install `nltk`
-----

If you finish the first two tasks, work on making sure that you have `nltk` downloaded and accessible to your jupyter notebooks. While you will not be allowed to use `nltk` for most of your homework, we will use it frequently in class to demonstrate tools.

[`nltk`](https://www.nltk.org/) (natural language toolkit) is a python package that comes with many useful implementations of NLP tools and datasets.

From the command line, using pip: `pip3 install nltk` or `pip install nltk`

[installing nltk](https://www.nltk.org/install.html)

In [103]:
import nltk
# for the tokenizers that we're going to use
# won't cause an error if you've already downloaded it
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [104]:
example = "N.K. Jemison is a science fiction author. Prof. Felix isn't."
words = nltk.word_tokenize(example)
print(words)

['N.K', '.', 'Jemison', 'is', 'a', 'science', 'fiction', 'author', '.', 'Prof.', 'Felix', 'is', "n't", '.']


In [105]:
moby_nltk_tokens = nltk.word_tokenize(contents)
# feel free to add/edit code
print(moby_nltk_tokens)
print("Token Count: ", len(moby_nltk_tokens))
print("Vocabulary count: ", len(set(moby_nltk_tokens)))

['\ufeffThe', 'Project', 'Gutenberg', 'eBook', 'of', 'Moby-Dick', ';', 'or', 'The', 'Whale', ',', 'by', 'Herman', 'Melville', 'This', 'eBook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'United', 'States', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', '.', 'You', 'may', 'copy', 'it', ',', 'give', 'it', 'away', 'or', 're-use', 'it', 'under', 'the', 'terms', 'of', 'the', 'Project', 'Gutenberg', 'License', 'included', 'with', 'this', 'eBook', 'or', 'online', 'at', 'www.gutenberg.org', '.', 'If', 'you', 'are', 'not', 'located', 'in', 'the', 'United', 'States', ',', 'you', 'will', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using', 'this', 'eBook', '.', 'Title', ':', 'Moby-Dick', ';', 'or', 'The', 'Whale', 'Author', ':', 'Herman', 'Melville', 'Release', 'Date', ':', 'June', ',', '2001', '[', 'eBook', '#', '2701', ']

1. How does the size of the vocabulary for Moby Dick compare when you use `nltk`'s tokenizer vs. the one that you made? __The size of my vocabulary is larger than the one that I get after using 'nltk'__

Task 4: Determining Sentiment
----

In [106]:
# we'll use nltk to access the reviews that we want to classify eventually
import nltk
import nltk.corpus as corpus

In [107]:
def load_word_list(filename):
    """
    Loads a lexicon from a plain text file in the format of one word per line.
    Parameters:
    filename (str): path to file

    Returns:
    list: list of words
    """
    with open(filename, 'r', encoding="utf-8") as f:
        # skip the header content
        for line in f:
            if line.strip() == "":
                break
        # read the rest of the lines into a list
        return [line.strip() for line in f]
    # otherwise return an empty list
    return []

In [108]:
# load in the positive and negative word lists here
# TODO: the paths to your negative/positive word files here
pos_lex = load_word_list("C:\\NEU\\NLP\\Lab1\\NLP Lab 1\\NLP Lab 1\\positive_words.txt")
neg_lex = load_word_list("C:\\NEU\\NLP\\Lab1\\NLP Lab 1\\NLP Lab 1\\negative_words.txt")

# TODO: How many words are in each list?
print(pos_lex)
print("Number of positive words:", len(set(pos_lex)))

print(neg_lex)
print("Number of negative words:", len(set(neg_lex)))

# TODO: Use python's list slicing to look at the first 10 elements in each list
print("\nFirst 10 elements in the positive word list are:", pos_lex[:10])
print("First 10 elements in the negative word list are:", neg_lex[:10])

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous', 'advantageously', 'advantages', 'adventuresome', 'adventurous', 'advocate', 'advocated', 'advocates', 'affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative', 'affluence', 'affluent', 'afford', 'affordable', 'affordably', 'afordable', 'agile', 'agilely', 'agility', 'agreeable', 'ag

In [109]:
# TODO: which words are in both the positive and the negative lists?
common_words = set(pos_lex).intersection(set(neg_lex))
print("Words common to both positive and negative listes are: ", common_words)
print("Number of common words are: ", len(common_words))

Words common to both positive and negative listes are:  {'enviousness', 'enviously', 'envious'}
Number of common words are:  3


Now, we'll create our rule-based classifier! We have access to the word lists that you loaded and anything else you know about the world (reflect on how you as a human being can tell if a review is positive/negative). Your classifier need not be perfect, but it should be reasonable (don't just say everything is positive!).

In [110]:
def rule_based_classify(tokens, pos_lexicon, neg_lexicon, verbose = False):
    """
    This function classifies a given tokenized text as positive or negative
    based on the provided lexicons.
    Parameters:
    tokens (list) - list of strings tokenized words in the text to classify
    pos_lexicon (list) - list of strings words in the positive word lexicon
    neg_lexicon (list) - list of strings words in the negative word lexicon
    verbose (boolean) - flag indicating whether or not to print verbose (debugging) output.
            Default value False.
    Returns:
    string "pos" if the list of tokens is positive overall, "neg" if they are negative overall.
    """
    # TODO: implement this function! This is our classifier.
    pos_count = sum(1 for token in tokens if token in pos_lexicon)
    neg_count = sum(1 for token in tokens if token in neg_lexicon)

    # Determine the overall sentiment based on the counts
    if pos_count > neg_count:
        result = "pos"
    elif neg_count > pos_count:
        result = "neg"
    else:
        result = "neutral"

    if verbose:
        print(f"Positive Count: {pos_count}, Negative Count: {neg_count}")
    
    return result

In [111]:
# now, we'll test out your classifier!
# Here are two example movie reviews.
nltk.download('movie_reviews')
movies = corpus.movie_reviews

# load in a single negative review
negative_toks = movies.words('neg/cv001_19502.txt')
# uncomment the text below to see the contents of the review
neg_text = " ".join(negative_toks)
print("The negative review is:", neg_text)

# load in a single positive review
positive_toks = movies.words('pos/cv992_11962.txt')
pos_text = " ".join(positive_toks)
print("The positive review is:", pos_text)


# TODO:
classification_result1 = rule_based_classify(negative_toks, pos_lex, neg_lex, verbose=False)
print("Overall Sentiment for negative review:", classification_result1)

classification_result2 = rule_based_classify(positive_toks, pos_lex, neg_lex, verbose=False)
print("Overall Sentiment for positive review:", classification_result2)

# Does our classification function label them correctly? Why or why not?
# take a look at the contents of the reviews

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...


The negative review is: the happy bastard ' s quick movie review damn that y2k bug . it ' s got a head start in this movie starring jamie lee curtis and another baldwin brother ( william this time ) in a story regarding a crew of a tugboat that comes across a deserted russian tech ship that has a strangeness to it when they kick the power back on . little do they know the power within . . . going for the gore and bringing on a few action sequences here and there , virus still feels very empty , like a movie going for all flash and no substance . we don ' t know why the crew was really out in the middle of nowhere , we don ' t know the origin of what took over the ship ( just that a big pink flashy thing hit the mir ) , and , of course , we don ' t know why donald sutherland is stumbling around drunkenly throughout . here , it ' s just " hey , let ' s chase these people around with some robots " . the acting is below average , even from the likes of curtis . you ' re more likely to get 

[nltk_data]   Package movie_reviews is already up-to-date!


1. What labels does your classifier assign these two reviews? __It assigns a 'pos' label for both the reviews__
2. Are these correct? __Its is classifying the negative review as positive as its based on a simple rule based classification.__

Task 5: How good is your sentiment classifier?
-----

Given the movies dataset from `nltk`, how many of the reviews does your classifier classify correctly?

We'll look at three different metrics: __accuracy__, __precision__, and __recall__.

__accuracy__: what you think of when you think of correctness.
$$ \frac{\texttt{number correct}}{\texttt{total number}}$$

Precision and recall require differentiated between the ways in which the classifier can be correct or incorrect.

- __true positive__: an example whose gold label is positive and that the classifier labels as positive
- __true negative__: an example whose gold label is negative and that the classifier labels as negative
- __false positive__: an example whose gold label is negative and that the classifier labels as positive
- __false negative__: an example whose gold label is positive and that the classifier labels as negative

In [112]:
import random
# you can use numpy's random functionality if you'd like to
# import numpy as np

In [113]:
# To see the available file ids, this is one way that we can access them.
# This will give you a list of neg/positive file ids.
print(len(movies.fileids('neg')))
# choose 100 random items without replacement from a list
print(random.sample(movies.fileids('neg'), 100))
print(len(movies.fileids('pos')))

1000
['neg/cv914_2856.txt', 'neg/cv588_14467.txt', 'neg/cv250_26462.txt', 'neg/cv649_13947.txt', 'neg/cv186_2396.txt', 'neg/cv558_29376.txt', 'neg/cv987_7394.txt', 'neg/cv377_8440.txt', 'neg/cv923_11951.txt', 'neg/cv100_12406.txt', 'neg/cv506_17521.txt', 'neg/cv358_11557.txt', 'neg/cv270_5873.txt', 'neg/cv706_25883.txt', 'neg/cv584_29549.txt', 'neg/cv900_10800.txt', 'neg/cv501_12675.txt', 'neg/cv447_27334.txt', 'neg/cv133_18065.txt', 'neg/cv905_28965.txt', 'neg/cv170_29808.txt', 'neg/cv009_29417.txt', 'neg/cv355_18174.txt', 'neg/cv526_12868.txt', 'neg/cv605_12730.txt', 'neg/cv408_5367.txt', 'neg/cv161_12224.txt', 'neg/cv455_28866.txt', 'neg/cv005_29357.txt', 'neg/cv926_18471.txt', 'neg/cv974_24303.txt', 'neg/cv134_23300.txt', 'neg/cv462_20788.txt', 'neg/cv144_5010.txt', 'neg/cv701_15880.txt', 'neg/cv827_19479.txt', 'neg/cv398_17047.txt', 'neg/cv891_6035.txt', 'neg/cv635_0984.txt', 'neg/cv180_17823.txt', 'neg/cv128_29444.txt', 'neg/cv516_12117.txt', 'neg/cv220_28906.txt', 'neg/cv762_156

In [114]:
# TODO:
# Write code that uses your classifier to classify 100 randomly chosen
# negative reviews and 100 randomly chosen positive reviews
# count the number of true positives, true negatives, false positives, and false negatives

# to get the tokens associated with a certain file id,
# tokens = movies.words(file_id)

# takes a long time to run if you loop over all fileids as opposed to just
# 100 randomly chosen ones
# make sure you don't classify the same review twice!
# (it takes us about 10 seconds to classify 200 reviews on a 2020 macbook air)
import random
from nltk.corpus import movie_reviews

# Assuming you have defined your rule_based_classify function with pos_lex and neg_lex
# pos_lex and neg_lex should be loaded or defined before using this code

# Get a list of negative and positive file IDs
neg_fileids = movie_reviews.fileids('neg')
pos_fileids = movie_reviews.fileids('pos')

# Choose 100 random file IDs without replacement for both negative and positive reviews
random_neg_fileids = random.sample(neg_fileids, 100)
random_pos_fileids = random.sample(pos_fileids, 100)

# Initialize counters for true positives, true negatives, false positives, and false negatives
true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0

# Classify negative reviews
for file_id in random_neg_fileids:
    tokens = movie_reviews.words(file_id)
    classification_result = rule_based_classify(tokens, pos_lex, neg_lex, verbose=False)
    
    # Update counters based on the ground truth (negative reviews)
    if 'neg' in file_id:
        if classification_result == 'neg':
            true_negatives += 1
        else:
            false_positives += 1

# Classify positive reviews
for file_id in random_pos_fileids:
    tokens = movie_reviews.words(file_id)
    classification_result = rule_based_classify(tokens, pos_lex, neg_lex, verbose=False)
    
    # Update counters based on the ground truth (positive reviews)
    if 'pos' in file_id:
        if classification_result == 'pos':
            true_positives += 1
        else:
            false_negatives += 1

# Print the results
print("True Positives:", true_positives)
print("True Negatives:", true_negatives)
print("False Positives:", false_positives)
print("False Negatives:", false_negatives)

# TODO: print out the number of true positives, false positives,
# false negatives, and true negatives


True Positives: 57
True Negatives: 79
False Positives: 21
False Negatives: 43


Here are the equations for accuracy, precision, and recall in terms of what we've just been counting. $tp$ means true positive, $fp$ means false positive, $fn$ means false negative, and $tn$ means true negative.

$$ accuracy = \frac{tp + tn}{tp + fp + fn + tn}$$

$$ precision = \frac{tp}{tp + fp}$$

$$ recall = \frac{tp}{tp + fn}$$

You can think of precision as "how many of my positive guesses were correct?" and recall as "how many of the positive examples did I find?" 😄

In [115]:
# TODO: calculate and print accuracy
accuracy = (true_positives+true_negatives)/(true_positives+true_negatives+false_positives+false_negatives)
print("The accuracy is:", accuracy)

The accuracy is: 0.68


In [116]:
# TODO: calculate and print precision
precision = true_positives/(true_positives+false_positives)
print("The precision is: ", precision)

The precision is:  0.7307692307692307


In [117]:
# TODO: calculate and print recall
recall = true_positives/(true_positives+false_negatives)
print("The recall is: ", recall)

The recall is:  0.57


Task 6: n-gram math
----



In [118]:
import time
from collections import Counter

def count_list(ls: list) -> dict:
    counts = {}
    for item in ls:
        # we're not going to be clever about counting here,
        # no conditionals, no sets, nothing
        counts[item] = ls.count(item)
    return counts

# see the difference between the following two items
example = [random.randint(0, 100) for i in range(2000)]
start = time.time()
count_list(example)
end = time.time()
print("That took:", end - start, "seconds!")

# this takes a very similar amount of time to count_dict from HW 1
start = time.time()
Counter(example)
end = time.time()
print("That took:", end - start, "seconds!")

That took: 0.02053046226501465 seconds!
That took: 0.0 seconds!


In [119]:
# TODO: put your create_ngrams (or make_ngrams) function here!
def make_ngrams(tokens: list, n: int) -> list:
    """Creates n-grams for the given token sequence.
    Args:
    tokens (list): a list of tokens as strings
    n (int): the length of n-grams to create

    Returns:
    list: list of tuples of strings, each tuple being one of the individual n-grams
    """
    # TODO: implement this function!
    ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
    #print(ngrams)
    return ngrams

In [120]:
# TODO: calculate the bigram score of the following sequence of tokens
# for this example, we'll use a "vanilla" scoring technique
# no Laplace smoothing, no unknown tokens

training_data = ["<s>", "I", "love", "dogs", "</s>", "<s>", "I", "love", "cats", "</s>", "<s>", "I", "love", "dinosaurs", "</s>"]

# TODO: call your create_ngrams function to get your bigrams

bigrams = make_ngrams(training_data, 2)

to_score = ["<s>", "I", "love", "cats", "</s>"]
start = time.time()

# BEGIN SCORING SECTION
# start probability at one so that we can multiply the probability of
# each subsequent next token with it

    # TODO: YOUR SCORE CALCULATION CODE HERE
total_prob = 1

for i in range(1, len(to_score)):
    bigram_count = bigrams.count((to_score[i-1], to_score[i]))
    unigram_count = training_data.count(to_score[i-1])

        # Vanilla scoring without Laplace smoothing
    probability = bigram_count / unigram_count if unigram_count > 0 else 0
    total_prob = total_prob*probability
# END SCORING SECTION
end = time.time()

# print your final probability
print("Final probability:", total_prob)
print("That took", end - start, "seconds!")


Final probability: 0.3333333333333333
That took 0.001001119613647461 seconds!


In [121]:
# Finally, pretend that we had a lot more data
training_data = ["<s>", "I", "love", "dogs", "</s>", "<s>", "I", "love", "cats", "</s>", "<s>", "I", "love", "dinosaurs", "</s>"]
# this is the amount of training data in the berp set(https://www1.icsi.berkeley.edu/Speech/berp.html)
training_data = training_data * 3778

# TODO: call your create_ngrams function here
bigrams = make_ngrams(training_data, 2)
to_score = ["<s>", "I", "love", "cats", "</s>"]

print("Number of training tokens:", len(training_data))
start = time.time()
# and what if we had 5000 sentences to score?
for example_num in range(5000):
    # TODO: COPY AND PASTE YOUR SCORING CODE HERE (between "BEGIN SCORING SECTION" and "END SCORING SECTION")
    # (remove any print statements that you have)
    # (make sure it is appropriately indented)
    total_prob = 1

    for i in range(1, len(to_score)):
        bigram_count = bigrams.count((to_score[i-1], to_score[i]))
        unigram_count = training_data.count(to_score[i-1])

        # Vanilla scoring without Laplace smoothing
        probability = bigram_count / unigram_count if unigram_count > 0 else 0
        total_prob = total_prob*probability

end = time.time()
print("That took", end - start, "seconds!")

Number of training tokens: 56670
That took 16.60546326637268 seconds!


What's the moral of the story? If you perform your counts at the same time you score, you'll be doing the same work over and over again which will result in a significantly slower model!

Make sure that you're gathering the counts that you need in `train` and only performing scoring calculations (as opposed to also counting things) in `score`.

This is particularly important when using larger data sets! (berp is not that big)

Task 7: Calculating tf-idf
----

Recall our discussion of tf-idf vector representations of documents in a corpus. Here, we will learn how to compute the tf-idf values for each token in a document. To calculate tf-idf, we'll need to first construct a term-document matrix from our data.

In [122]:
import numpy as np

In [123]:
def tf(term: str, document: list) -> float:
    """
    Calculate term frequency
    Parameters:
    term - string
    document - list of strings (tokenized document)
    Return:
    float term frequency
    """
    return np.log10(document.count(term) + 1)

def idf(N: int, df_t: int) -> float:
    """
    Calculate inverse document frequency given the
    number of documents and the number of documents the term appears in.
    Paramenters:
    N - int (number of documents)
    df_t - int (number of documents the term appears in)
    Return:
    float inverse document frequency
    """
    if df_t == 0:
        # Handle the case where the term is not present in any document
        return 0.0
    else:
        return np.log10(N / df_t)


def term_in_documents(term: str, documents: list) -> int:
    """
    Calculate the number of documents in a list of documents that a target
    term appears in.
    Parameters:
    term - str
    documents - list of list of str (list of tokenized documents)
    Return:
    int number of documents the term appears in
    """
    #TODO: IMPLEMENT ME
    count = 0
    for document in documents:
        if term in document:
            count += 1
    return count


# load in the data
def load_tokens(filename):
    f = open(filename, 'r', encoding="utf-8")
    contents = f.read().lower()
    f.close()
    # if you don't have nltk installed, use another tokenization
    # strategy here like str.split()
    return nltk.word_tokenize(contents)

mobydick = load_tokens("C:\\NEU\\NLP\\Lab1\\NLP Lab 1\\NLP Lab 1\\mobydick.txt")
shakes = load_tokens('C:\\NEU\\NLP\\Lab1\\NLP Lab 1\\NLP Lab 1\\shakespeare.txt')
pandp = load_tokens('C:\\NEU\\NLP\\Lab1\\NLP Lab 1\\NLP Lab 1\\prideandprejudice.txt')
books = [mobydick, shakes, pandp]

In [124]:
# we won't create the entire term-document matrix, we'll just do it for a few key terms that we
# care about for the sake of time
# TODO: pick 3 - 5 words
words = ['jupiter', 'beauty', 'rose', 'parsley', 'good']

In [125]:
# create the term-document matrix
matrix = []

# iterate through your chosen words
for word in words:
    # count how many times it occurs in each book
    # this is just for debugging for you
    counts = [str(b.count(word)) for b in books]
    print(word, ":\t", "\t".join(counts))


    # calculate the term frequency for each book
    # for this word
    # tf_words should be the same length as counts
    # tf_words = YOUR CODE HERE
    tf_words = [tf(word, book) for book in books]
    # calculate idf for this term
    # this will be a single scalar
    N = len(books)
    df_t = term_in_documents(word, books)
#     idf_t = YOUR CODE HERE
    idf_t = idf(N, df_t)
    # multiply tf with idf for each book/each
    # term frequency you calculated
#     tfidf_words = YOUR CODE HERE
    tfidf_words = [tf * idf_t for tf in tf_words]
    # add the tfidf numbers to your matrix
    matrix.append(tfidf_words)

    # uncomment to see visually the different components (helpful for debugging)
    print(word, " tf:\t", "\t".join([str(x) for x in tf_words]))
    print(word, "idf:\t", idf_t)
    print(word, " tf-idf:\t", "\t".join([str(x) for x in tfidf_words]))

# if you'd like to, uncomment the following code to make the matrix into a numpy array
matrix = np.array(matrix)

print("Term-Document Matrix:")
print(matrix)

jupiter :	 4	34	0
jupiter  tf:	 0.6989700043360189	1.5440680443502757	0.0
jupiter idf:	 0.17609125905568124
jupiter  tf-idf:	 0.12308250810568454	0.2718968859972835	0.0
beauty :	 8	228	26
beauty  tf:	 0.9542425094393249	2.359835482339888	1.4313637641589874
beauty idf:	 0.0
beauty  tf-idf:	 0.0	0.0	0.0
rose :	 23	64	7
rose  tf:	 1.380211241711606	1.8129133566428555	0.9030899869919435
rose idf:	 0.0
rose  tf-idf:	 0.0	0.0	0.0
parsley :	 1	1	0
parsley  tf:	 0.3010299956639812	0.3010299956639812	0.0
parsley idf:	 0.17609125905568124
parsley  tf-idf:	 0.053008750949996714	0.053008750949996714	0.0
good :	 195	2793	180
good  tf:	 2.292256071356476	3.446226401778163	2.2576785748691846
good idf:	 0.0
good  tf-idf:	 0.0	0.0	0.0
Term-Document Matrix:
[[0.12308251 0.27189689 0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.05300875 0.05300875 0.        ]
 [0.         0.         0.        ]]


1. What should the dimensions of your matrix be? __5*3__
2. What happens if you attempt to calculate tfidf of a term that exists in *none* of your books? __It gives the error of division by zero. I have modified the code to return the value as zero instead of throwing an error.__
2. What happens if you attempt to calculate tfidf of a term that exists in *all* of your books? __The tf values in such case are high, but the idf values are zero.__

In [133]:
# check the dimensions of your matrix
# number of rows should match number of words
# number of cols should match number of documents (books)
print(matrix.shape)

(5, 3)


In [127]:
def cosine_sim(v1: list, v2: list) -> float:
    """
    Calculate the cosine similarity between two vectors of the same length.
    Parameters:
    v1 - list (of numbers)
    v2 - list (of numbers)
    Return:
    float cosine similarity
    """
    # you may find the numpy functions np.dot() and np.linalg.norm() useful
    # TODO: implement
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    
    if norm_v1 == 0 or norm_v2 == 0:
        return 0  # Avoid division by zero
    
    similarity = dot_product / (norm_v1 * norm_v2)
    return similarity

# calculate the similarity between two *word* vectors
# we'll just do word vectors because unless matrix is a numpy array
# it is (more) difficult to get column vectors

word_vectors = {
    'tiger': np.array([0.2, 0.8, -0.3]),
    'lion': np.array([0.0, 0.0, -0.6]),
    'cat': np.array([-0.4, 0.3, 0.7])
}
similarity_tiger_lion = cosine_sim(word_vectors['tiger'], word_vectors['lion'])
print("Cosine Similarity between 'tiger' and 'lion':", similarity_tiger_lion)

Cosine Similarity between 'tiger' and 'lion': 0.3418817293789138


Optional Section (no extra credit)
----

In [128]:
# if you finish, which book is closest to moby dick?
# you'll need a *column* vector here (instead of a row vector)


In [129]:
# if you finish finish, which book is closest to moby dick, but remake your matrix with all vocabulary terms?
# create the term-document matrix
# you'll want to use counters for each book's vocabulary for the sake of efficiency


# you may want to re-define new counter versions of your tf, idf, term_in_documents
# functions so that they work with counters


In [130]:
# verify the shape of your matrix

In [131]:
# which book is actually closest to moby dick?